# Leverage using PBP stats method

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.linear_model import RidgeCV, Ridge
from joblib import Parallel, delayed, parallel_backend

data_DIR = "../data/rapm/"
export_DIR = "./fdata/"

fig_DIR = "../figs/rapm/"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
league = "nba"
margin = -5 # from the perspective of the team with the ball
seconds_remaining = 60 # This is time remaining at the start of the possession
pre_game_win_prob = 0.55 # 55% to win pregame
end_of_possession_seconds_remaining = 45 # seconds remaining at end of possession

In [ ]:
url = f"https://api.pbpstats.com/get-leverage/{league}/{margin}/{pre_game_win_prob}/{seconds_remaining}/{end_of_possession_seconds_remaining}"

response = requests.get(url)
response_json = response.json()
response_json

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
url = "https://www.rotowire.com/betting/nba/tables/games-archive.php"
response = requests.get(url,headers=header)
odds_js = response.json()

In [ ]:
odds = pd.DataFrame(odds_js)

In [ ]:
odds.to_parquet("odds_2024.parquet")

In [ ]:


# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.layer1 = nn.Linear(3, 12)
        self.layer2 = nn.Linear(12, 12)
        self.output = nn.Linear(12, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

# Initialize the model
model = PyTorchModel()

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

# Prepare your data
inputs = torch.tensor(df[['lead', 'net', 'time']].values, dtype=torch.float32)
targets = torch.tensor(df['win'].values, dtype=torch.float32).unsqueeze(1)

dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Print the model summary
print(model)
